### Formula 1 Machine Learning Model

** Please be sure to run `pip install fastf1` before running**


In [3]:
import fastf1
import pandas as pd
from IPython.display import display
from datetime import datetime, timezone

years = [2024, 2025]
required_sessions = ['Session4DateUtc', 'Session5DateUtc']  # Qualifying & Race
cache_dir = 'cache'

fastf1.Cache.enable_cache(cache_dir)
fastf1.set_log_level('WARNING')

now = datetime.now(timezone.utc)

# This will hold all merged result DataFrames
all_combined = []

for year in years:
    print(f"\n=== Processing season {year} ===")
    try:
        schedule = fastf1.get_event_schedule(year, include_testing=True)
    except Exception as e:
        print(f"Could not fetch schedule for {year}: {e}")
        continue

    # Filter to only those events where both Quali & Race have completed
    completed = []
    for _, evt in schedule.iterrows():
        ok = True
        for key in required_sessions:
            if key not in evt or pd.isna(evt[key]):
                ok = False
                break
            # ensure timezone-aware
            sess_time = evt[key]
            if sess_time.tzinfo is None:
                sess_time = sess_time.replace(tzinfo=timezone.utc)
            if sess_time >= now:
                ok = False
                break
        if ok:
            completed.append(evt)

    if not completed:
        print(f"No fully completed events found for {year}.")
        continue

    df_events = pd.DataFrame(completed)

    # Pull Qualifying & Race results for each completed event
    for _, evt in df_events.iterrows():
        name = evt['EventName']
        print(f"\n— {year} {name} —")
        try:
            # Qualifying
            q = fastf1.get_session(year, name, 'Q')
            q.load()
            q_res = q.results[['Abbreviation', 'Position', 'Q1', 'Q2', 'Q3']].copy()
            q_res.rename(columns={
                'Position':    'QualiPosition',
                'Q1':          'QualiQ1',
                'Q2':          'QualiQ2',
                'Q3':          'QualiQ3'
            }, inplace=True)
        except Exception as e:
            print(f"Qualifying failed for {name}: {e}")
            continue

        try:
            # Race
            r = fastf1.get_session(year, name, 'R')
            r.load()
            r_res = r.results[['Abbreviation', 'Position', 'GridPosition', 'Time', 'Status', 'Points']].copy()
            r_res.rename(columns={
                'Position':   'RacePosition',
                'Time':       'RaceTime'
            }, inplace=True)
        except Exception as e:
            print(f"Race failed for {name}: {e}")
            continue

        # Merge on driver abbreviation
        merged = pd.merge(q_res, r_res, on='Abbreviation', how='inner')
        merged['Year']      = year
        merged['EventName'] = name
        all_combined.append(merged)

# Final merge
if all_combined:
    final_df = pd.concat(all_combined, ignore_index=True)
    display(
        final_df[[
            'Year','EventName','Abbreviation',
            'QualiPosition','QualiQ1','QualiQ2','QualiQ3',
            'RacePosition','GridPosition','RaceTime','Status','Points'
        ]]
        .sort_values(['Year','EventName','QualiPosition'])
    )
else:
    print("No data could be pulled for any of the specified seasons.")



=== Processing season 2024 ===

— 2024 Bahrain Grand Prix —

— 2024 Saudi Arabian Grand Prix —


_api        WARNING 	Failed to align laps for drivers: ['10']
_api        WARNING 	Driver 55: Car data is incomplete!
_api        WARNING 	Driver 38: Car data is incomplete!
_api        WARNING 	Driver 55: Position data is incomplete!
_api        WARNING 	Driver 38: Position data is incomplete!
_api        WARNING 	Driver 241: Position data is incomplete!
_api        WARNING 	Driver 242: Position data is incomplete!
_api        WARNING 	Driver 243: Position data is incomplete!



— 2024 Australian Grand Prix —

— 2024 Japanese Grand Prix —

— 2024 Chinese Grand Prix —


_api        WARNING 	Driver 241: Position data is incomplete!
_api        WARNING 	Driver 242: Position data is incomplete!
_api        WARNING 	Driver 243: Position data is incomplete!
core        WARNING 	Driver 1 completed the race distance 00:08.313000 before the recorded end of the session.



— 2024 Miami Grand Prix —


...

logger      WARNING 	Failed to load session info data!
core        WARNING 	Failed to load extended driver information!
logger      WARNING 	Failed to load result data from Ergast!
core        WARNING 	No result data for this session available on Ergast! (This is expected for recent sessions)
core        WARNING 	Failed to load driver list and session results!
logger      WARNING 	Failed to load session status data!
logger      WARNING 	Failed to load track status data!
logger      WARNING 	Failed to load timing data!
logger      WARNING 	Failed to load telemetry data!
logger      WARNING 	Failed to load weather data!
logger      WARNING 	Failed to load race control messages!
logger      WARNING 	Failed to load session info data!
core        WARNING 	Failed to load extended driver information!
logger      WARNING 	Failed to load result data from Ergast!
core        WARNING 	No result data for this session available on Ergast! (This is expected for recent sessions)
core        WARNING 	


=== Processing season 2025 ===


logger      WARNING 	Failed to load schedule from FastF1 backend!
logger      WARNING 	Failed to load schedule from F1 API backend!
logger      WARNING 	Failed to load schedule from Ergast API backend!


⚠️  Could not fetch schedule for 2025: Failed to load any schedule data.


,Year,EventName,Abbreviation,QualiPosition,QualiQ1,QualiQ2,QualiQ3,RacePosition,GridPosition,RaceTime,Status,Points
40,2024,Australian Grand Prix,VER,1.0,0 days 00:01:16.819000,0 days 00:01:16.387000,0 days 00:01:15.915000,19.0,1.0,NaT,Retired,0.0
41,2024,Australian Grand Prix,SAI,2.0,0 days 00:01:16.731000,0 days 00:01:16.189000,0 days 00:01:16.185000,1.0,2.0,0 days 01:20:26.843000,Finished,25.0
42,2024,Australian Grand Prix,PER,3.0,0 days 00:01:16.805000,0 days 00:01:16.631000,0 days 00:01:16.274000,5.0,6.0,0 days 00:00:56.309000,Finished,10.0
43,2024,Australian Grand Prix,NOR,4.0,0 days 00:01:17.430000,0 days 00:01:16.750000,0 days 00:01:16.315000,3.0,3.0,0 days 00:00:05.904000,Finished,15.0
44,2024,Australian Grand Prix,LEC,5.0,0 days 00:01:16.984000,0 days 00:01:16.304000,0 days 00:01:16.435000,2.0,4.0,0 days 00:00:02.366000,Finished,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...
194,2024,Spanish Grand Prix,MAG,16.0,0 days 00:01:12.937000,NaT,NaT,17.0,16.0,0 days 00:00:54.788000,Lapped,0.0
195,2024,Spanish Grand Prix,TSU,17.0,0 days 00:01:12.985000,NaT,NaT,19.0,17.0,0 days 00:01:08.904000,Lapped,0.0
196,2024,Spanish Grand Prix,RIC,18.0,0 days 00:01:13.075000,NaT,NaT,15.0,18.0,0 days 00:00:44.050000,Lapped,0.0
197,2024,Spanish Grand Prix,ALB,19.0,0 days 00:01:13.153000,NaT,NaT,18.0,20.0,0 days 00:00:58.149000,Lapped,0.0
